In [1]:
import face_recognition
import cv2
import os
import glob
import numpy as np

class SimpleFacerec:
    def __init__(self): # self = javadaki this , bu fonksiyon parametre alamamakta
        self.known_face_encodings = [] # boş list tannımladık
        self.known_face_names = []

        # Resize frame for a faster speed
        self.frame_resizing = 0.25

    def load_encoding_images(self, images_path):
        """
        Load encoding images from path
        :param images_path:
        :return:
        """
        # Load Images
        images_path = glob.glob(os.path.join(images_path, "*.*")) # images_path bir dizedir

        print("{} encoding images found.".format(len(images_path)))

        # Store image encoding and names
        for img_path in images_path:
            img = cv2.imread(img_path)
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Get the filename only from the initial file path.
            basename = os.path.basename(img_path)
            (filename, ext) = os.path.splitext(basename)
            # Get encoding
            img_encoding = face_recognition.face_encodings(rgb_img)[0]

            # Store file name and file encoding
            self.known_face_encodings.append(img_encoding)
            self.known_face_names.append(filename)
        print("Encoding images loaded")

    def detect_known_faces(self, frame):
        small_frame = cv2.resize(frame, (0, 0), fx=self.frame_resizing, fy=self.frame_resizing)
        # Find all the faces and face encodings in the current frame of video
        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances) # argmin min değerin  indexsini dönderir
            if matches[best_match_index]:
                name = self.known_face_names[best_match_index]
            face_names.append(name)

        # Convert to numpy array to adjust coordinates with frame resizing quickly
        face_locations = np.array(face_locations)
        face_locations = face_locations / self.frame_resizing
        return face_locations.astype(int), face_names


#### os.path.basename()
os.path.basename() fonksiyonu, bir dosya yolunun son bölümünü (dosya adını) almak için kullanılır.

#### images_path
glob.glob() fonksiyonu tarafından belirli bir desene uyan dosya adlarını içeren bir liste dizesidir. Dolayısıyla, bu ifade, belirli bir dizinde bulunan dosya sayısını elde eder.

#### glob

txt_files = glob.glob('*.txt') Şu anki dizindeki tüm .txt dosyalarını listeler                                     
 *.* deseni, tüm dosya uzantılarını içeren dosya adlarını eşleştirir.

#### img_encoding = face_recognition.face_encodings(rgb_img)[0]
Bu satır, face_recognition kütüphanesinin face_encodings fonksiyonunu kullanarak bir görüntünün yüz kodlamasını (yüz özelliklerini temsil eden bir sayısal vektör) elde etmek için kullanılır.

Yüz kodlaması, bir yüzün belirli özelliklerini ifade eden bir sayısal vektördür. Bu özellikler, yüzün gözleri, burunu, ağzı gibi belirli noktaları ve bu noktalardaki gölgelerin yoğunlukları gibi detaylardır. Bu vektör, yüzlerin benzersiz özelliklerini temsil eder ve yüz tanıma sistemlerinde kullanılarak farklı yüzlerin tanınmasına yardımcı olur.

Örneğin, face_recognition.face_encodings(rgb_img) ifadesi, rgb_img olarak adlandırılan bir RGB görüntüsündeki yüzleri kodlamak için kullanılır. Bu ifadenin döndürdüğü değer, bir veya birden fazla yüz kodlaması içeren bir liste olacaktır. [0] indeksi, bu liste içindeki ilk yüz kodlamasını alır.

Yani, img_encoding değişkenine atanan değer, rgb_img görüntüsündeki ilk yüzün kodlamasıdır. Bu kodlama, daha sonra yüzlerin tanınması için kullanılabilir veya kaydedilebilir.

#### small_frame = cv2.resize(frame, (0, 0), fx=self.frame_resizing, fy=self.frame_resizing)
(0, 0) değeri görüntünün hedef boyutunu belirtmek için kullanılmaz, aslında boyutun otomatik olarak hesaplanmasını sağlar. Bu, görüntünün orijinal boyutunun korunmasını sağlar.

fx ve fy parametreleri, görüntünün boyutunu manuel olarak belirlemek için kullanılır. Bu parametreler, görüntünün genişlik ve yükseklik boyutunu sırasıyla genişletmek ve daraltmak için kullanılır. Örneğin, fx=0.5 ve fy=0.5 parametreleri, orijinal görüntünün genişliğinin ve yüksekliğinin yarısı kadar olan bir boyutta yeni bir görüntü oluşturur.

#### face_locations = face_recognition.face_locations(rgb_small_frame)
top: Dikdörtgenin sol üst köşesinin y koordinatı.                                                               
right: Dikdörtgenin sağ üst köşesinin x koordinatı.                                                            
bottom: Dikdörtgenin sağ alt köşesinin y koordinatı.                                                             
left: Dikdörtgenin sol alt köşesinin x koordinatı.

#### face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

face_recognition.face_encodings() fonksiyonu, bir görüntüdeki belirli yüzlerin kodlamalarını (yüz özelliklerini temsil eden bir sayısal vektör) elde etmek için kullanılır. Bu fonksiyon, bir RGB görüntüsü ve yüzlerin koordinatlarını alır ve bu yüzlerin kodlamalarını döndürür.

Bu kod parçasında, rgb_small_frame adlı bir görüntü ve face_locations adlı bir liste verilir. face_locations listesi, görüntüdeki yüzlerin konumlarını içerir. Yani, face_locations listesindeki her bir yüzün konumu, rgb_small_frame görüntüsünde belirlenmiş olan yüzlerdir.

face_recognition.face_encodings() fonksiyonu, bu yüz konumlarını kullanarak her bir yüzün kodlamasını hesaplar. Sonuç olarak, bu fonksiyon, yüzlerin kodlamalarını içeren bir liste döndürür. Her bir kodlama, yüzün belirli özelliklerini ifade eden bir sayısal vektördür.     
                                                                              Bu kod, rgb_small_frame görüntüsündeki face_locations listesinde belirtilen yüzlerin kodlamalarını hesaplar ve face_encodings adlı bir liste olarak döndürür. Bu liste, her bir yüzün kodlamasını içerir.

#### face_locations
face_locations, face_recognition.face_locations() fonksiyonundan dönen bir liste tipindedir. Bu liste, görüntüdeki yüzlerin konumlarını içerir. Her bir yüzün konumu, bir tuple (demet) olarak temsil edilir.

Her bir tuple, yüzün dört köşesinin koordinatlarını içerir. Bu koordinatlar, genellikle (top, right, bottom, left) veya (y1, x2, y2, x1) şeklinde sıralanır. Koordinatlar, yüzün dikdörtgen sınırlayıcı kutusunun sol üst ve sağ alt köşelerini temsil eder.

Örneğin, bir face_locations listesi aşağıdaki gibi olabilir:

python
Copy code
face_locations = [(62, 275, 127, 209), (110, 385, 175, 320)]
Bu liste, iki yüzün konumunu içerir. İlk yüzün koordinatları (62, 275, 127, 209) ve ikinci yüzün koordinatları (110, 385, 175, 320) olarak belirtilir. Her bir tuple, bir yüzün sol üst ve sağ alt köşelerinin koordinatlarını içerir.



#### face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
face_distances = [0.534, 0.723, 0.812]
Bu liste, verilen yüz kodlamasının, bilinen yüz kodlamaları listesi içindeki üç yüz kodlaması ile olan benzerliklerini içerir. İlk değer (0.534) verilen yüz kodlamasının ilk bilinen yüz kodlaması ile olan benzerliğini, ikinci değer (0.723) verilen yüz kodlamasının ikinci bilinen yüz kodlaması ile olan benzerliğini, vb. ifade eder.

Bu benzerlik değerleri, yüz tanıma sistemlerinde bir eşik değeri olarak kullanılabilir. Örneğin, belirli bir eşik değerinin altındaki benzerlik değerlerine sahip yüzler tanınmamış olarak kabul edilebilir.